In [1]:
import requests
import json
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec


jikan_api_json = requests.get('https://raw.githubusercontent.com/jikan-me/jikan-rest/master/storage/api-docs/api-docs.json')

raw_jikan_api_spec = json.loads(jikan_api_json.content)

jikan_api_spec = reduce_openapi_spec(raw_jikan_api_spec)

In [2]:
import os
from langchain.llms import AzureOpenAI
from langchain.agents.agent_toolkits.openapi import planner


os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2022-12-01"
os.environ["OPENAI_API_BASE"] = ""
os.environ["OPENAI_API_KEY"] = "<api_key>"

llm = AzureOpenAI(
    deployment_name="dv3",
    model_name="text-davinci-003"
)

In [3]:
from langchain.requests import RequestsWrapper


requests_wrapper = RequestsWrapper()

jikan_agent = planner.create_openapi_agent(jikan_api_spec, requests_wrapper, llm)

In [5]:
user_query = ("List the titles of the top 10 anime released in 2009. Order the list by score and sort it in descending order")

jikan_agent.run(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to get the list of top 10 anime released in 2009 sorted by score in descending order
Observation:  1. GET /top/anime with query params of year=2009 and sort=score:desc
2. GET /anime/{id}/full to get the full details of the top 10 anime.
Thought: I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /top/anime with query params of year=2009 and sort=score:desc
2. GET /anime/{id}/full to get the full details of the top 10 anime.

> Entering new AgentExecutor chain...
I need to first get the top 10 anime according to 2009 year and score descending.
Action: requests_get
Action Input: 
{
  "url": "https://api.jikan.moe/v4/top/anime",
  "params": {
    "year": 2009,
    "sort": "score:desc"
  },
  "output_instructions": "Extract the ids from the response for each anime."
}
Observation: 5114
Thought:I need to use the ids of each anime to get the full details f

"The titles of the top 10 anime released in 2009, sorted by score in descending order are: \n1. Fullmetal Alchemist: Brotherhood\n2. Code Geass: Hangyaku no Lelouch\n3. Gintama°\n4. Steins;Gate\n5. Gintama'\n6. Hunter x Hunter (2011)\n7. Kimi no Na wa.\n8. Hajime no Ippo: The Fighting!\n9. Clannad: After Story\n10. Cowboy Bebop"